In [3]:
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession
    .builder
    .appName('predict_churn_rate')
    .getOrCreate()
    )

In [3]:
train_df = spark.read.option('header', 'true').csv('D:/kkbox-churn-prediction-challenge/data/churn_comp_refresh/train_v2.csv', inferSchema=True)
members_df = spark.read.option('header', 'true').csv('D:/kkbox-churn-prediction-challenge/data/churn_comp_refresh/members_v3.csv', inferSchema=True)
transactions_df = spark.read.option('header', 'true').csv('D:/kkbox-churn-prediction-challenge/data/churn_comp_refresh/transactions_v2.csv', inferSchema=True)
user_logs_df = spark.read.option('header', 'true').csv('D:/kkbox-churn-prediction-challenge/data/churn_comp_refresh/user_logs_v2.csv', inferSchema=True)

In [4]:
train_df = pd.read_csv('D:/kkbox-churn-prediction-challenge/data/churn_comp_refresh/train_v2.csv')
members_df = pd.read_csv('D:/kkbox-churn-prediction-challenge/data/churn_comp_refresh/members_v3.csv')
transactions_df = pd.read_csv('D:/kkbox-churn-prediction-challenge/data/churn_comp_refresh/transactions_v2.csv')
user_logs_df = pd.read_csv('D:/kkbox-churn-prediction-challenge/data/churn_comp_refresh/user_logs_v2.csv')

In [6]:
print('train_df : ', train_df.shape)
print('members_df : ', members_df.shape)
print('transactions_df : ', transactions_df.shape)
print('user_logs_df : ', user_logs_df.shape)

train_df :  (970960, 2)
members_df :  (6769473, 6)
transactions_df :  (1431009, 9)
user_logs_df :  (18396362, 9)


### train_df 살펴보기

In [7]:
train_df.columns

Index(['msno', 'is_churn'], dtype='object')

In [8]:
(train_df['is_churn'] == 0).sum(), (train_df['is_churn'] == 1).sum()

(883630, 87330)

In [10]:
train_df.isnull().sum()

msno        0
is_churn    0
dtype: int64

In [12]:
len(train_df['msno'].unique())

970960